In [22]:
import pandas as pd
import numpy as np

In [23]:
Train = pd.read_excel('df_final.xlsx')

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
Train[['월', '화', '수', '목', '금', '토','일', 
    'month', 'day', 'hour', 'resting_day', 'working_day', 
    '가구', '가전', '건강기능', '농수축', '생활용품', '속옷', '의류', '이미용', '잡화', '주방', '침구',
    '가을', '겨울', '봄', '여름']] = Train[['월', '화', '수', '목', '금', '토','일', 
    'month', 'day', 'hour', 'resting_day', 'working_day', 
    '가구', '가전', '건강기능', '농수축', '생활용품', '속옷', '의류', '이미용', '잡화', '주방', '침구',
    '가을', '겨울', '봄', '여름']] * 1

In [26]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36899 entries, 0 to 36898
Data columns (total 48 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          36899 non-null  int64         
 1   20분당시청률             36899 non-null  float64       
 2   방송일시                36899 non-null  datetime64[ns]
 3   노출(분)               36899 non-null  float64       
 4   마더코드                36899 non-null  int64         
 5   상품코드                36899 non-null  int64         
 6   상품명                 36899 non-null  object        
 7   상품군                 36899 non-null  object        
 8   판매단가                36899 non-null  int64         
 9   취급액                 36899 non-null  int64         
 10  주문량                 36899 non-null  float64       
 11  월                   36899 non-null  int32         
 12  화                   36899 non-null  int32         
 13  수                   36899 non-null  int32     

In [27]:
x = Train.drop(['취급액','상품군','주문량','상품명','Unnamed: 0','방송일시','20분당시청률_Band',
                'season'],axis = 1)
y = Train['취급액']

In [28]:
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size = 0.3, random_state = 924)

In [29]:
xTrain.shape

(25829, 40)

In [30]:
yTrain.shape

(25829,)

In [31]:
xTest.shape

(11070, 40)

In [32]:
yTest.shape

(11070,)

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
myparam = { 
'n_estimators':[21,51,101,201,],
'min_samples_split':[2,4,6,8,10],
'max_depth':[3,5,7,9,11,13]}

In [42]:
rfModel =  RandomForestRegressor(n_estimators=51,
                              max_depth=13,
                              min_samples_split=10,
                              n_jobs=2,
                              random_state=42)

In [35]:
gridCV = GridSearchCV(rfModel, param_grid = myparam, cv = 5, n_jobs = 3)
gridCV.fit(xTrain,yTrain)
print(gridCV.best_params_)
gridCV.best_score_

{'max_depth': 13, 'min_samples_split': 10, 'n_estimators': 51}


0.525978317379622

In [43]:
rfModel.fit(xTrain, yTrain)

RandomForestRegressor(max_depth=13, min_samples_split=10, n_estimators=51,
                      n_jobs=2, random_state=42)

In [44]:
pred = rfModel.predict(xTest)

In [45]:
pred.shape
rfModel.score(xTrain,yTrain)
# 0.65

0.6544279638800792

In [46]:
y_True = yTest.to_numpy()
y_True

array([ 4842000, 24908000,  2837000, ..., 40553000, 32439000,  3782000],
      dtype=int64)

In [47]:
pred

array([12480185.48369479, 37418703.67121711,  8108287.65488197, ...,
       41478085.25206506, 20076657.77400089, 11798727.16823836])

In [48]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

mean_absolute_percentage_error(yTest,pred)

89.66108884213662